# Output Ordering

**Задача:**
1. найти 20 редких терминов (10 со статьей на википедии и 10 без);
2. получить для их текстов выдачу классификатора;
3. добавить к выдаче доп. информацию (id статьи, № предложения, есть/нет маркер и тип маркера);
4. отсортировать выдачу по этой информации.

In [1]:
import os
import re

from collections import defaultdict

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.corpus import stopwords
from razdel import sentenize

import pymorphy2 as pm2

In [2]:
import gc
import re
from collections import Counter

import evaluate
from datasets import ClassLabel, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    DataCollatorWithPadding,
    TrainingArguments, 
    Trainer,
    pipeline
)
from transformers.pipelines.pt_utils import KeyDataset

import torch

import numpy as np
import pandas as pd

import pymorphy2 as pm2
from razdel import sentenize
from nltk import word_tokenize

from ipymarkup import show_span_box_markup
from ipymarkup.palette import palette, RED, GREEN
from tqdm.notebook import tqdm

In [3]:
SEED = 42
MAX_SEQUENCE_LEN = 512

### 0. Manual Term Search

In [4]:
# terms with related wiki article
wiki_terms = [
    ('акустический', 'эмиссия'),
    ('квантовый', 'точка'),
    ('внутренний', 'волна'),
    ('волна', 'рэлей'),
    ('калибровочный', 'бозон'),
    ('меченый', 'атом'),
    ('синглётный', 'кислород'),
    ('число', 'стокс'),
    ('число', 'рэлей'),
    ('тяжёлый', 'вода'),
]

# terms without article
non_wiki_terms = [
    ('волна', 'накачка'),
    ('динамический', 'рекристаллизация'),
    ('ионосферный', 'турбулентность'),
    ('квантовый', 'биение'),
    ('оптический', 'выпрямление'),
    ('паразитный', 'мода'),
    ('электронный', 'концентрация'),
    ('хаотический', 'синхронизация'),
    ('функция', 'память'),
    ('ударный', 'слой'),
]

# combined terms
terms = wiki_terms + non_wiki_terms

len(terms)

20

### 1. Data Crawling

In [5]:
def get_or_cache_token_parse(lemmatizer, token, cache):
    """Extracts token parse from cache, or adds it to the latter if not found."""
    if token in cache:
        token_parse = cache[token]
    else:
        token_parse = lemmatizer.parse(token)[0]
        cache[token] = token_parse

    return token_parse

In [6]:
lemmatizer = pm2.MorphAnalyzer(lang='ru')
pm2_cache = {}

Загрузим датасет - корпус научных статей по физике:

In [7]:
data_directory = '../data/elibrary_physics_15k'
article_id = re.compile(r'_([0-9]+).htm$')

file_contents = {}
for filename in tqdm(os.scandir(data_directory), desc='Loading data'):
    if filename.is_file():
        file_name = filename.name
        file_id = int(article_id.search(file_name).groups(1)[0])
        
        with open(filename, "r", encoding='utf-8') as f:
            text = f.read()
            file_contents[file_id] = text

print(f'Размер корпуса: {len(file_contents)} статей')

Loading data: 0it [00:00, ?it/s]

Размер корпуса: 15000 статей


Удалим из корпуса статьи, написанные не на русском языке.

In [8]:
MIN_CYRILLIC_THRESHOLD = 0.2

cyrillic_sym = re.compile(r'[а-яёА-ЯЁ]')

Алгоритм отсеивания статей по языку:
1. подсчитывается кол-во кириллических символов в статье;
2. если соотношение кириллицы >= MIN_CYRILLIC_THRESHOLD -> статья русскоязычная.

In [9]:
rus_articles = {}

for id, article in tqdm(file_contents.items(), desc='Filtering out non-russian articles'):
    rus_count = 0
    for _ in cyrillic_sym.finditer(article):
        rus_count += 1

    if rus_count >= len(article) * MIN_CYRILLIC_THRESHOLD:
        rus_articles[id] = article

print(f'Удалено нерусскоязычных статей: {len(file_contents) - len(rus_articles)}')

Filtering out non-russian articles:   0%|          | 0/15000 [00:00<?, ?it/s]

Удалено нерусскоязычных статей: 4062


In [10]:
articles_count = len(rus_articles)
print(f'Кол-во русскоязычных статей: {articles_count}')

Кол-во русскоязычных статей: 10938


Напишем вспомогательные функции для анализа лемм:

In [11]:
def match_bigram(word1_parse, word2_parse):
    """
    Tries to match bigram of w1 and w2 with any of the patterns.
    If succeed, returns a tuple ((w1_norm, w2_norm), pattern).
    """
    word1_pos = word1_parse.tag.POS
    word2_pos = word2_parse.tag.POS

    # failed pattern matching
    if word2_pos != 'NOUN' or word1_pos not in {'NOUN', 'ADJF'}:
        return None

    word1_inflection = (word1_parse.tag.gender, word1_parse.tag.number, word1_parse.tag.case)
    word2_inflection = (word2_parse.tag.gender, word2_parse.tag.number, word2_parse.tag.case)

    # 'adj + noun' pattern
    if (
        word1_pos == 'ADJF' and word2_pos == 'NOUN' 
        and word1_inflection[1] == word2_inflection[1]
        and word1_inflection[2] == word2_inflection[2]
        and (
            word1_inflection[0] == word2_inflection[0]
            or word1_inflection[0] is None and word1_inflection[1] == 'plur'
        )
    ):
        return (word1_parse.normal_form, word2_parse.normal_form), 'прил. + сущ.'

    # 'noun + noun' pattern
    if word1_pos == 'NOUN' and word2_pos == 'NOUN' and word2_inflection[2] == 'gent':
        return (word1_parse.normal_form, word2_parse.normal_form), 'сущ. + сущ.'

    return None

In [12]:
def simplify_bigram(bigram, lemmatizer, cache):
    """Simplifies bigram and returns its normal form."""
    unigram_1, unigram_2 = tuple(bigram.split(sep='_'))

    normgram_1 = get_or_cache_token_parse(lemmatizer, unigram_1, cache).normal_form
    normgram_2 = get_or_cache_token_parse(lemmatizer, unigram_2, cache).normal_form

    return f'{normgram_1}_{normgram_2}'

Зададим список стоп-слов:

In [13]:
stop_words = stopwords.words('russian')

stop_words.extend(
    [
        # adjective-like pronouns
        'мой', 'твой', 'ваш', 'наш', 'свой', 'его', 'ее', 'их',
        'тот', 'этот', 'такой', 'таков', 'сей', 'который',
        'весь', 'всякий', 'сам', 'самый', 'каждый', 'любой', 'иной', 'другой',
        'какой', 'каков', 'чей', 'никакой', 'ничей',
        'какой-то', 'какой-либо', 'какой-нибудь', 'некоторый', 'некий',
        # participles
        'соответствующий', 'следующий', 'данный',
        # numerals
        'один',
        # insignificant words
        'друг',
    ]
)

stop_words = set(stop_words)

Зададим список стоп-биграмм:

In [14]:
stop_bigrams = []

stop_bigrams.extend(
    [
        ('крайний', 'мера'), ('сегодняшний', 'день'), ('настоящий', 'время'), ('настоящий', 'работа'),
        ('настоящий', 'статья'), ('точка', 'зрение'), ('первый', 'очередь'), ('последний', 'год'),
    ]
)

stop_bigrams = set(stop_bigrams)

Подходящие слова должны содержать только буквы и дефисы (последние не в начале/конце слова):

In [15]:
def match_unigram(word):
    """Checks if a word is long enough and contains only letters and dashes."""
    if word.startswith('-') or word.endswith('-'):
        return False

    word_dashless = word.replace('-', '')
    return len(word_dashless) >= 3 and word_dashless.isalpha()

Загрузим датасет предложений с посчитанными метриками:

In [16]:
bigram_mi3_df = pd.read_csv('../data/data_frames/bigram_tfidf_mi3_scores_full.csv', index_col=0)

In [17]:
def has_predicate(lemmatizer, cache, tokens, predicates):
    """Checks if the sentence contains a predicate."""
    for token in tokens:
        token_pos = get_or_cache_token_parse(lemmatizer, token, cache).tag.POS
        if token_pos in predicates:
            return True

    return False

In [18]:
def is_definition(bigram, tokens):
    """Checks if the sentence has a structure <... bigram ... - ...>."""
    hyphens = ('-', '−', '–', '—')
    for x in hyphens:
        if x in tokens:
            split_idx = tokens.index(x)
            pre_define = tokens[:split_idx]

            if bigram[0] in pre_define and bigram[1] in pre_define:
                bigram_end_idx = tokens.index(bigram[1])

                if split_idx - bigram_end_idx <= 7:
                    return True

    return False

In [19]:
def has_hyphen(tokens):
    """Checks if the sentence has a hyphen and is valid."""
    valid_tokens_count = 0
    
    for token in tokens:
        if len(token) > 2:
            valid_tokens_count += 1
    if valid_tokens_count < 3:
        return False

    hyphens = ('-', '−', '–', '—')
    for x in hyphens:
        if x in tokens:
            return True

    return False

Наложим ограничения на попадание в выборку:

In [20]:
def get_suitability_status(sentence, formulae_pattern, external_link, internal_link, good_symbols):
    """Checks all conditions to include the sentence in the dataset."""
    if (
        # no equations
        formulae_pattern.search(sentence) is None
        # no external links
        and external_link.search(sentence) is None
        # no internal links
        and internal_link.search(sentence) is None
        # no incomplete fragments
        and len(sentence) >= 10
    ):
        bad_symbols = set(sentence.lower()) - good_symbols

        if not bad_symbols:
            # remove the mess in the beginning
            first_letter = re.search(r'[А-ЯA-Z]', sentence)
            if first_letter is not None:
                return first_letter.start()

    return -1

In [21]:
subtitle_pattern = re.compile('(==.*==(\\n)+)+')
clumped_pattern = re.compile('[а-яА-Я0-9]\.[а-яА-Я0-9]')

formulae_pattern = re.compile(r'\n.\n.\n')
external_link_pattern = re.compile(r'\[([0-9])+\]')
internal_link_pattern = re.compile(r'\(([0-9])+\)')

In [22]:
predicates = {'INFN', 'VERB', 'PRTS', 'ADJS'}

Отберем все символы, не встречающиеся в формулах:

In [23]:
good_symbols = {
    'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 
    'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r',
    's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    '(', ')', '%', '\n', ' ', ',', '.', '"', '\'', '!', '/', ':', ';', '?', '[', ']', '«', '»',
    '³', '—', '–', '−', '-', '…', 
}

Извлечем слова-маркеры:

In [24]:
markers_file = '../data/relation_markers/markers.txt'

In [25]:
markers_to_block = {}

with open(markers_file, 'r') as f:
    while (line := f.readline()) != '\n':
        line = line.strip()

        # block headline
        if line[0] == '*' and line[1] != '*':
            cur_block = line[1:]
        # block separator
        elif line[0] == '*' and line[1] == '*':
            continue
        # marker word
        else:
            markers_to_block[line.lower()] = cur_block

In [26]:
markers_to_block

{'это': 'Определения',
 'означает': 'Определения',
 'вид': 'Определения',
 'тип': 'Определения',
 'разновидность': 'Определения',
 'определить': 'Определения',
 'определять': 'Определения',
 'определение': 'Определения',
 'вариант': 'Определения',
 'представлять': 'Определения',
 'термин': 'Определения',
 'содержат': 'Структура',
 'компонент': 'Структура',
 'часть': 'Структура',
 'входит в состав': 'Структура',
 'структура': 'Структура',
 'конструкция': 'Структура',
 'включить': 'Структура',
 'включать': 'Структура',
 'состоять': 'Структура',
 'модуль': 'Структура',
 'элемент': 'Структура',
 'принадлежать': 'Структура',
 'состав': 'Структура',
 'использоваться': 'Использование',
 'использовать': 'Использование',
 'использование': 'Использование',
 'применение': 'Использование',
 'применяться': 'Использование',
 'позволить': 'Использование',
 'позволять': 'Использование',
 'давать возможность': 'Использование',
 'дать возможность': 'Использование',
 'удобный инструмент': 'Использование'

In [27]:
block_importance = {
    'Определения': 1,
    'Структура': 2,
    'Использование': 3,
    'Отличительные особенности': 4,
    'Дополнительные сведения': 5,
    None: 6,
}

In [28]:
def extract_sentences(text, text_idx, bigram):
    """Extracts classifiable sentences for target bigram from text."""
    sentences = []
    for sentence_idx, substr in enumerate(sentenize(text)):
        sentence = substr.text.strip(r'\n')
        
        # removing subtitles
        subtitle = subtitle_pattern.search(sentence)
        if subtitle is not None:
            sentence = sentence[:subtitle.start(0)] + sentence[subtitle.end(0):]
        
        # capital letter check
        if not sentence[0].isupper():
            continue
        
        # removing sentences with newline characters
        if sentence.find('\n') != -1:
            continue

        # removing clumped sentences
        if clumped_pattern.search(sentence) is not None:
            continue

        # deemphasize sentence
        sentence = sentence.replace('́', '')
        
        tokens = word_tokenize(sentence.lower(), language='russian')

        # saving only sentences with target bigram
        has_target_bigram = False
        normalized_tokens = [get_or_cache_token_parse(lemmatizer, x, pm2_cache).normal_form for x in tokens]
        for i in range(len(normalized_tokens) - 1):
            if normalized_tokens[i] == bigram[0] and normalized_tokens[i + 1] == bigram[1]:
                has_target_bigram = True
                break
        if not has_target_bigram:
            continue
        
        # removing sentences without predicates
        has_predicates = True
        
        standardized_tokens = []
        for token in tokens:
            if match_unigram(token):
                standardized_tokens.append(token)

        if (
            not has_hyphen(tokens)
            and not has_predicate(lemmatizer, pm2_cache, standardized_tokens, predicates)
        ):
            has_predicates = False

        # removing formulas and links
        sentence_start = get_suitability_status(sentence, formulae_pattern, external_link_pattern, internal_link_pattern, good_symbols)
        if sentence_start != -1 and has_predicates:
            final_sentence = sentence[sentence_start:]

            # searching for markers
            marker_type = None

            # trying to find them in normal sentence
            final_tokens = word_tokenize(final_sentence.lower(), language='russian')
            token_set = set(final_tokens)
            for marker, block in markers_to_block.items():
                if marker in token_set:
                    marker_type = block
                    break

            # checking the definition pattern
            normalized_sentence = [get_or_cache_token_parse(lemmatizer, x, pm2_cache).normal_form for x in final_tokens]
            if is_definition(bigram, normalized_sentence):
                marker_type = 'Определения'

            marker_importance = block_importance[marker_type]

            sentences.append([final_sentence, text_idx, (sentence_idx + 1), marker_type, marker_importance])

    return sentences

Сохраним подходящие контексты:

In [29]:
articles_df = {}

for term in tqdm(terms, desc='Saving matching contexts'):
    articles = bigram_mi3_df[(bigram_mi3_df['unigram_1'] == term[0]) & (bigram_mi3_df['unigram_2'] == term[1])]['articles_id_list']
    articles = eval(list(articles.values)[0])

    matched = []
    for article_id in articles:
        text = rus_articles[article_id]
        matched.extend(extract_sentences(text, article_id, term))

    df = pd.DataFrame(matched, columns=['sentence', 'article_id', 'sentence_number', 'marker_type', 'marker_importance'])
    articles_df[f'{term[0]}_{term[1]}'] = df

Saving matching contexts:   0%|          | 0/20 [00:00<?, ?it/s]

In [30]:
for term, df in articles_df.items():
    df.to_csv(f'../data/data_frames/eval_20/{term}.csv')

### 2. Sentence Classification

In [31]:
id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

In [32]:
checkpoint_dir = '../data/train_results/bert_pair/ruRoberta-large/outputs/checkpoint-4233'

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint_dir,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, device='cuda', batch_size=32)

In [33]:
for term, df in tqdm(articles_df.items(), desc='Finding fitting sentences for term abstracts'):
    # adding bigram info to dataframe
    cleaned_term = term.replace('_', ' ')

    df['text_pair'] = cleaned_term
    df.rename(columns={'sentence': 'text'}, inplace=True)
    df = df.reindex(columns=['text', 'text_pair', 'article_id', 'sentence_number', 'marker_type', 'marker_importance'])

    # preparing HF dataset
    ds = Dataset.from_pandas(df)
    ds = ds.remove_columns(['article_id', 'sentence_number', 'marker_type', 'marker_importance'])

    # classifying dataset items
    pos_sentences = []
    pair_generator = (pair for pair in ds)
    for idx, output in enumerate(classifier(pair_generator)):
        if output['label'] == 'positive':
            text = ds[idx]['text']
            score = output['score']

            # restoring extra info
            extra_info = list(df[df['text'] == text].values[0][2:])

            pos_sentences.append([text] + extra_info + [score])

    # sorting outputs by importance
    sorted_sentences = sorted(pos_sentences, key=lambda x: (-x[4], x[5]), reverse=True)

    # saving results
    final_df = pd.DataFrame(sorted_sentences, columns=['sentence', 'article_id', 'sentence_number', 'marker_type', 'marker_importance', 'score'])
    final_df.to_csv(f'../data/data_frames/eval_20_results/{term}.csv')

Finding fitting sentences for term abstracts:   0%|          | 0/20 [00:00<?, ?it/s]

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\logging\__init__.py", line 1100, in emit
    msg = self.format(record)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\logging\__init__.py", line 943, in format
    return fmt.format(record)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\logging\__init__.py", line 678, in format
    record.message = record.getMessage()
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\logging\__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "D

In [34]:
model.cpu()
del model, classifier

gc.collect()
torch.cuda.empty_cache()

### 3. Notes

1. на некоторые термины без вики-статьи можно найти статьи с похожими названиями, но они о других явлениях;
2. выдача по терминам неравномерная: кол-во найденных предложений варьируется в среднем от 3 до 10, но бывают и исключения (квантовая точка - 59);
3. сортировал выдачу сначала по убыванию важности маркера (определения и структрура "важнее" доп. сведений), а после по убыванию оценки классификатора;
4. маркеры не всегда адекватно структурируют выдачу - например, в определения проскакивают предложения, не определяющие свойства целевого термина, но содержащие слова-триггеры "это", "является" и т.д.